In [1]:
import pickle
import collections

sensor_data = {
    "sideways": [
        pickle.load(open("sensor_data_pos_0_sideways.pickle", "rb")),
        pickle.load(open("sensor_data_pos_1_sideways.pickle", "rb")),
        pickle.load(open("sensor_data_pos_2_sideways.pickle", "rb")),
        pickle.load(open("sensor_data_pos_3_sideways.pickle", "rb")),
        pickle.load(open("sensor_data_pos_4_sideways.pickle", "rb")),
        pickle.load(open("sensor_data_pos_5_sideways.pickle", "rb")),
    ],
    "upways": [
        pickle.load(open("sensor_data_pos_0_upways.pickle", "rb")),
        pickle.load(open("sensor_data_pos_1_upways.pickle", "rb")),
        pickle.load(open("sensor_data_pos_2_upways.pickle", "rb")),
        pickle.load(open("sensor_data_pos_3_upways.pickle", "rb")),
        pickle.load(open("sensor_data_pos_4_upways.pickle", "rb")),
        pickle.load(open("sensor_data_pos_5_upways.pickle", "rb")),
    ],
}

In [2]:
%matplotlib widget

from test_get_bs_geometry import run_get_bs_geometry_proxy
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import matplotlib.pyplot as plt
import numpy as np
import math

bs_geoms = run_get_bs_geometry_proxy(sensor_data["upways"][4])
print(bs_geoms[1].origin)

# Plot upways vectors for position 5

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

ax.set_xlim3d(-4, 4)
ax.set_ylim3d(-4, 4)
ax.set_zlim3d(-4, 4)

ax.scatter(0, 0, 0, marker='o', s=100)
ax.text(0, 0, 0, "CF", size=20, zorder=1)

for bs_id, geom in bs_geoms.items():
    
    # Plot point
    point = geom.origin
    ax.scatter(*point, marker='o', s=100)
    ax.text(*point, f"{bs_id}", size=20, zorder=1)
    
    # Plot rotation
    r_matrix = geom.rotation_matrix
    # X
    ax.plot3D([point[0], point[0] + r_matrix[0, 0]], [point[1], point[1] + r_matrix[1, 0]], [point[2], point[2] + r_matrix[2, 0]], 'red')
    # Y
    ax.plot3D([point[0], point[0] + r_matrix[0, 1]], [point[1], point[1] + r_matrix[1, 1]], [point[2], point[2] + r_matrix[2, 1]], 'green')
    # Z
    ax.plot3D([point[0], point[0] + r_matrix[0, 2]], [point[1], point[1] + r_matrix[1, 2]], [point[2], point[2] + r_matrix[2, 2]], 'blue')


ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()


[ 0.02684959 -1.57676054  1.22299617]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
bs_geoms = run_get_bs_geometry_proxy(sensor_data["sideways"][1], sideways=True)

# Plot sideways vectors for position

def bs_is_on_floor(bs_id):
    if bs_id == 2:
        return True
    else:
        return False
    

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

ax.set_xlim3d(-4, 4)
ax.set_ylim3d(-4, 4)
ax.set_zlim3d(-4, 4)

ax.scatter(0, 0, 0, marker='o', s=100)
ax.text(0, 0, 0, "CF", size=20, zorder=1)

for bs_id, geom in bs_geoms.items():
    
    # Plot position
    point = geom.origin
    
    ax.scatter(*point, marker='o', s=100)
    ax.text(*point, f"{bs_id}", size=20, zorder=1)
    
    # Plot rotation
    r_matrix = geom.rotation_matrix

    # X
    ax.plot3D([point[0], point[0] + r_matrix[0, 0]], [point[1], point[1] + r_matrix[1, 0]], [point[2], point[2] + r_matrix[2, 0]], 'red')
    # Y
    ax.plot3D([point[0], point[0] + r_matrix[0, 1]], [point[1], point[1] + r_matrix[1, 1]], [point[2], point[2] + r_matrix[2, 1]], 'green')
    # Z
    ax.plot3D([point[0], point[0] + r_matrix[0, 2]], [point[1], point[1] + r_matrix[1, 2]], [point[2], point[2] + r_matrix[2, 2]], 'blue')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()

Adjusting BS estimation results for sideways CF...
Adjusting BS 1...
Adjusting BS estimation results for sideways CF...


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
import math
import numpy as np
from scipy.spatial.transform import Rotation
from collections import namedtuple

System = namedtuple("System", ['P', 'R', 'bs'])

def gen_rot_x(angle):
    co = math.cos(angle)
    si = math.sin(angle)

    return np.array([
        [1, 0, 0],
        [0, co, -si],
        [0, si, co]])


def gen_rot_z(angle):
    co = math.cos(angle)
    si = math.sin(angle)

    return np.array([
        [co, -si, 0],
        [si, co, 0],
        [0, 0, 1]])

# Transform a point in ref frame x to ref frame r using transformation x->r
def transform_point_x_to_r(v_x, Px_r, Rx_r):
    return np.dot(Rx_r, v_x) + Px_r

# Transform a rotation in ref frame x to ref frame r using transformation x->r
def transform_rot_x_to_r(Rx_y, Ry_r):
    return np.dot(Ry_r, Rx_y)

# Transform a base station (y) in ref frame x to ref frame r using transformation x->r
def transform_x_to_r(Sy_x, Sx_r):
    Py_r = transform_point_x_to_r(Sy_x.P, Sx_r.P, Sx_r.R)
    Ry_r = transform_rot_x_to_r(Sy_x.R, Sx_r.R)
    return System(Py_r, Ry_r, Sy_x.bs)

# Transform a point in ref frame r to ref frame x using transformation x->r
def transform_point_to_x_from_r(v_r, Px_r, Rx_r):
    Rr_x = np.matrix.transpose(Rx_r)
    return np.dot(Rr_x, v_r - Px_r)

# Find the transformation x->r when we know the bs pos/rot for both x and r
def transform_from_ref_x_to_r_same_bs(Sb_x, Sb_r):
    Rb_x_inv = np.matrix.transpose(Sb_x.R)
    Rx_r = np.dot(Sb_r.R, Rb_x_inv)
    Px_r = Sb_r.P - np.dot(Rx_r, Sb_x.P)

    return System(Px_r, Rx_r, Sb_x.bs)


# Averaging of quaternions
# From https://stackoverflow.com/a/61013769
def q_average(Q, W=None):
    if W is not None:
        Q *= W[:, None]
    eigvals, eigvecs = np.linalg.eig(Q.T@Q)
    return eigvecs[:, eigvals.argmax()]

def system_average(S):
    bs = S[0].bs
    for s in S:
        if s.bs != bs:
            raise Exception("Different base stations")

    Q = map(lambda s : Rotation.from_matrix(s.R).as_quat(), S)
    q = q_average(np.array(list(Q)))
    r = Rotation.from_quat(q).as_matrix()

    P = map(lambda s : s.P, S)
    p = np.average(np.array(list(P)), axis=0)

    return System(p, r, bs)



position = 1

results_upways = run_get_bs_geometry_proxy(sensor_data["upways"][position])
results_sideways = run_get_bs_geometry_proxy(sensor_data["sideways"][position], sideways=True)
        

measurements = [
    [
        # The global ref frame - the origin
        System(geom.origin, geom.rotation_matrix, bs_id) for bs_id, geom in results_upways.items()
    ],
    [
        System(geom.origin, geom.rotation_matrix, bs_id) for bs_id, geom in results_sideways.items()
    ],
]

##################

def print_system(system):
    print(f"Base station {system.bs} @ {system.P}")


def probe_position(Sbs_ref, Sbs0_other, Sbs1_other):
    # Find transform from other ref frame to global using bs0
    Sother_g = transform_from_ref_x_to_r_same_bs(Sbs0_other, Sbs_ref)

    # The meassurement of base station 1 in the other ref frame, converted to global
    Sbs1mOther_g = transform_x_to_r(Sbs1_other, Sother_g)

    return Sbs1mOther_g


print()
ref = measurements[0][0]

result = [ref]
found = [ref.bs]

not_done = True

while not_done:
    not_done = False
    samples = {}

    print("--- iteration")

    for measurement in measurements:
        # Find a reference system in this measurement
        from_bs = None
        from_sys = None
        from_sys_g = None
        for system in measurement:
            bs = system.bs
            for sys in result:
                if sys.bs == bs:
                    from_bs = bs
                    from_sys = system
                    from_sys_g = sys
                    break
            if from_bs is not None:
                break

        # Transform all base stations in this measurement to
        # the global system, unless we already have a result for a
        # particular base station
        if from_bs is not None:
            print(f"Using {from_bs} as reference")
            for sys in measurement:
                if sys is not from_sys:
                    if sys.bs not in found:
                        s = probe_position(from_sys_g, from_sys, sys)
                        from_to = (from_bs, sys.bs)
                        if not from_to in samples:
                            samples[from_to] = []
                        samples[from_to].append(s)
        else:
            not_done = True

    # Average over all samples sets
    for from_to, sample_set in samples.items():
        # Averaging might create suboptimal solution
        new_sys = system_average(sample_set)
        result.append(new_sys)
        found.append(new_sys.bs)


for sys in result:
    print_system(sys)
    
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

ax.set_xlim3d(-4, 4)
ax.set_ylim3d(-4, 4)
ax.set_zlim3d(-4, 4)

ax.scatter(0, 0, 0, marker='o', s=100)
ax.text(0, 0, 0, "CF", size=20, zorder=1)

for sys in result:
    
    # Plot position
    point = sys.P
    ax.scatter(*point, marker='o', s=100)
    ax.text(*point, f"{sys.bs}", size=20, zorder=1)
    
    # Plot rotation
    r_matrix = sys.R
    # X
    ax.plot3D([point[0], point[0] + r_matrix[0, 0]], [point[1], point[1] + r_matrix[1, 0]], [point[2], point[2] + r_matrix[2, 0]], 'red')
    # Y
    ax.plot3D([point[0], point[0] + r_matrix[0, 1]], [point[1], point[1] + r_matrix[1, 1]], [point[2], point[2] + r_matrix[2, 1]], 'green')
    # Z
    ax.plot3D([point[0], point[0] + r_matrix[0, 2]], [point[1], point[1] + r_matrix[1, 2]], [point[2], point[2] + r_matrix[2, 2]], 'blue')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()

Adjusting BS estimation results for sideways CF...
Adjusting BS 1...
Adjusting BS estimation results for sideways CF...

--- iteration
Using 0 as reference
--- iteration
Using 0 as reference
Using 1 as reference
Base station 0 @ [0.71406854 1.36064821 1.15254272]
Base station 1 @ [ 0.4384016  -1.08545015  1.2356447 ]
Base station 2 @ [-0.1328311  -1.26662247 -1.09662177]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …